본 문제에 대한 주 학습 목표는 데이터에 존재하는 다양한 형태의 결측치(범주형 데이터 결측치, 수치형 데이터 결측치)를 다양한 방식으로 처리 및 비교하여, 평가하는 과정을 전략으로 삼아 수행하는 것이다.

# 결측치 처리 전략

1. 결측치가 존재하는 데이터 열을 삭제한다.

2. y값(생존여부)에 대한 각각의 값을 따로 평균하여 결측치를 전처리한다. (충돌 당시 속도)

3. 딥러

In [1]:
from tensorflow.keras.models import Sequential # Pre-ready model: Sequential
from tensorflow.keras. layers import Dense # for MLP
from keras.optimizers import SGD, Adam # optimizer
from sklearn.preprocessing import StandardScaler # 데이터 정규화
# 모델 학습 데이터 분리용
from sklearn.model_selection import train_test_split
# 모델 평가 지표
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 데이터 호출 및 전처리용 라이브러리
import pandas as pd
import numpy as np

# 정규화 도구 생성
stdsclr = StandardScaler() # 표준화를 위한 함수를 호출
stdsclr_goal = StandardScaler() # class 값의 표준화를 위한 함수를 호출

In [2]:
# 데이터 전처리
def preprocess(data):

  # 이진 데이터 mapping
  data["Gender"] = data["Gender"].map({"Female": 0, "Male": 1})
  data["Helmet_Used"] = data["Helmet_Used"].map({"No": 0, "Yes": 1})
  data["Seatbelt_Used"] = data["Seatbelt_Used"].map({"No": 0, "Yes": 1})

  # DataFrame 의 모든 열을 평균  = 0, 표준편차 = 1 인 표준정규분포로 표준화하는 코드
  # 그리고 fit 을 통해서 각각의 column 의 평균, 표준 편차를 계산하고, transform 통해서 각각의 평균, 표준편차로 변환
  data_norm = stdsclr.fit_transform(data)

  # 전처리 된 x 데이터 반환
  return data_norm.astype(float)

In [3]:
# 공통된 모델 예측 프로세스 실행 함수
def model_prediction(data):

  # 예측할 y데이터 입력
  y_data = data["Survived"]
  x_data = data.drop("Survived", axis=1)

  # 목표값 정규화
  goal = stdsclr_goal.fit_transform(y_data.values.reshape(-1, 1))

  # 이진 분류 전처리
  rawnp_norm = preprocess(x_data)

  # 학습용/테스트용 데이터 분리
  # rawnp_norm -> x, goal-> y 라고 명시해주고
  #test와 train 의 비율을 0.2로 설정
  # random state 는 항상 데이터가 동일하게 섞이도록 시드를 고정한다는 뜻
  x_train, x_test, y_train, y_test = train_test_split(
    rawnp_norm, goal, test_size=0.2, random_state=42
  )

  # 데이터를 input: 5, output: 1로 설정(활성화 함수: 시그모이드)
  model = Sequential()
  model.add(Dense(1, input_dim=5, activation='sigmoid'))

  # optimizer로 adam 설정(사용했던 sgd 주석처리)
  #sgd = SGD(learning_rate=0.1)
  adam = Adam(
      beta_1=0.9,
      beta_2=0.999
  )
  # 모델 컴파일 및 학습
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

  model.summary()
  model.fit(x_data, y_data, epochs=100)

  # 모델 예측
  preds = model.predict(x_test)
  # 예측된 모델 원래 값으로 복구
  preds = stdsclr_goal.inverse_transform(preds)

  # MAE로 평가
  mae_norm = mean_absolute_error(y_test, preds)
  print(f"테스트 MAE: {mae_norm:.4f}\n") # 소숫점 네자리까지 출력

  # RMSE로 평가
  rmse = np.sqrt(mean_squared_error(y_test, preds))
  print(f"테스트 RMSE: {rmse:.4f}\n") #소숫점 두자리까지 출력

  return preds

In [4]:
# 1. 결측치를 -1로 처리하고 학습

data1 = pd.read_csv("./accident.csv", header=0)
# 성별에 대한 데이터 결측치는 하나 밖에 없으므로, 이에 대한 데이터 행은 삭제
data1 = data1.dropna(subset='Gender')

# Nan의 값 처리가 힘들어 결측값을 -1로 변환
data1['Speed_of_Impact'] = data1['Speed_of_Impact'].fillna(-1)

# 모델 예측 및 평가
prediction_value = model_prediction(data1)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 1)                   │               6 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6 (24.00 B)

 Trainable params: 6 (24.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.5042 - loss: 29.8317
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5435 - loss: 26.8558 
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.4910 - loss: 28.8939
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.4911 - loss: 28.6121 
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.4856 - loss: 29.1922 
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.5023 - loss: 27.5505
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.4754 - loss: 28.8788
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5326 - loss: 25.0906 
Epoch 9/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.4837 - loss: 27.8625
Epoch 10/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4833 - loss: 26.7600  
Epoch 11/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4352 - loss: 30.0008 
Epoch 12/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accur

In [5]:
# 2. 결측치를 제거하고 학습

data2 = pd.read_csv("./accident.csv", header=0)
# 이전 처럼 Gender는 평가에서 제외하도록 함
data2 = data2.dropna(subset='Gender')

# 결측치 제거 전 데이터 shape(199, 6)
print(data2.shape)

# 결측치 제거
data2 = data2.dropna()

# 결측치 제거 전 데이터 shape(196, 6)
print(data2.shape)

# 모델 예측 및 평가
prediction_value = model_prediction(data2)

(199, 6)
(196, 6)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_1 (Dense)                      │ (None, 1)                   │               6 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6 (24.00 B)

 Trainable params: 6 (24.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4872 - loss: 41.2052  
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4843 - loss: 41.1390 
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4853 - loss: 40.3267 
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4767 - loss: 40.8271 
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4982 - loss: 39.3776 
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4951 - loss: 38.3794 
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5134 - loss: 36.8158 
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5013 - loss: 36.5540 
Epoch 9/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4732 - loss: 37.7783 
Epoch 10/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4607 - loss: 38.6178 
Epoch 11/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4853 - loss: 36.6486  
Epoch 12/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy:

In [6]:
# 3. 결측치를 평균 처리 한 후, 학습

data3 = pd.read_csv("./accident.csv")
# 이전 처럼 Gender는 평가에서 제외하도록 함
data3 = data3.dropna(subset='Gender')

# 결측치를 가진 행 15번째 인덱스의 결측치 확인
print(np.array(data3['Speed_of_Impact'][15]))
data3['Speed_of_Impact'] = data3['Speed_of_Impact'].fillna(np.mean(data3['Speed_of_Impact']))

# 결측치 평균 처리 후 행 15번째 인덱스의 값 확인
print(np.array(data3['Speed_of_Impact'][15]))

# 모델 예측 및 평가
prediction_value = model_prediction(data3)

nan
70.43367346938776


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                      │ (None, 1)                   │               6 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6 (24.00 B)

 Trainable params: 6 (24.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5388 - loss: 15.8878  
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4941 - loss: 17.7204 
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4997 - loss: 17.4408 
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4973 - loss: 16.8926 
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5112 - loss: 15.8959 
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4839 - loss: 16.9080 
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5173 - loss: 15.4307 
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5186 - loss: 14.3144 
Epoch 9/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4637 - loss: 15.8736 
Epoch 10/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5137 - loss: 13.1471 
Epoch 11/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5306 - loss: 13.6002 
Epoch 12/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
테스트 MAE: 1.0456

테스트 RMSE: 1.3036

